# Secondary structure calculations

Two possibilities:
* Nupack
* RNAfold

In [10]:
import pandas as pd

# Loading sequences

In [11]:
store = pd.HDFStore('100000_random_50nt_sequences.h5')

In [12]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: 100000_random_50nt_sequences.h5
/data            frame        (shape->[100000,2])

In [13]:
df = store['data']

In [14]:
store.close()

In [15]:
df.head()

,Bin,canonical
290826,14,TACTGATATACCTCTCCGTTAAGGATACGCCGTAAGCTCCATTGGT...
597807,10,CGAGCACGAATACTGGTTCACGAAGTAGGATTCCCATTTTAATCAG...
914453,11,AAATTAAGGCATACAGTAAACGACTTAAATCGAACACCGGTCTTGC...
377378,5,AAAGGGCAACTGTGGAAGCGGAGGGTAGACTGGTAAATAACATCGT...
1470839,9,CGCTAGATTTCACTGTGCTTTGGGTACGAAACCCAGCATATTCGCC...


In [16]:
df = df.reset_index()
df = df.drop('index', axis=1)

In [17]:
df.head()

,Bin,canonical
0,14,TACTGATATACCTCTCCGTTAAGGATACGCCGTAAGCTCCATTGGT...
1,10,CGAGCACGAATACTGGTTCACGAAGTAGGATTCCCATTTTAATCAG...
2,11,AAATTAAGGCATACAGTAAACGACTTAAATCGAACACCGGTCTTGC...
3,5,AAAGGGCAACTGTGGAAGCGGAGGGTAGACTGGTAAATAACATCGT...
4,9,CGCTAGATTTCACTGTGCTTTGGGTACGAAACCCAGCATATTCGCC...


In [18]:
# Taking only one sequence
seq = df.canonical.loc[0]

In [19]:
seq

'TACTGATATACCTCTCCGTTAAGGATACGCCGTAAGCTCCATTGGTCACA'

# Predicting secondary structure with Nupack

This is covered in "Complex Analysis" part of their manual. The manual is in "doc" folder supplied with source code.

## Testing NuPack installation

Installation on Ubuntu is straightforward according to their manual. I'm not covering it here

In [20]:
# Partition function
!pfunc -v

No input file specified.
Requesting input manually.
% NUPACK 3.1.0
% Program: pfunc
% Start time: Tue Apr 25 23:16:24 2017
% Command: pfunc -v 
Enter sequence: ^C


In [21]:
# Base Pairing
!pairs

No input file specified.
Requesting input manually.
Enter output file prefix: ^C


In [22]:
# Minimum free energy (MFE) secondary structure
!mfe

No input file specified.
Requesting input manually.
Enter output file prefix: ^C


In [23]:
# All secondary structures within specified minimum free energy gap
!subopt

No input file specified.
Requesting input manually.
Enter output file prefix: ^C


## In order to run nupack, one needs sequence saved in a file

I'm creating a short function to save sequence to file

In [24]:
def SeqToFile(seq):
    # Saving a single sequence to the temporary file, which is accepted by Nupack
    seq_file_handler = open('seq.in', 'w')
    seq_file_handler.write(seq)
    seq_file_handler.close()

In [25]:
# Saving sample sequence to input file
SeqToFile(seq)

In [26]:
# File is in local directory
!ls

100000_random_50nt_sequences.h5  seq.ppairs
Pandas_Indexing_Simple.ipynb	 Sequences_to_Sparse_Kmer_Matrix.ipynb
Secondary_Structure.ipynb	 Working_With_Sequences_K_Mers.ipynb
seq.in


## Calculating probability of each base to pair with other base

In [27]:
# -T 37 is how one specifies temperature
!pairs -T 37 seq

In [28]:
# The result appears in file seq.ppairs
!ls

100000_random_50nt_sequences.h5  seq.ppairs
Pandas_Indexing_Simple.ipynb	 Sequences_to_Sparse_Kmer_Matrix.ipynb
Secondary_Structure.ipynb	 Working_With_Sequences_K_Mers.ipynb
seq.in


In [29]:
!pairs -T 37 seq

## Reading results

In [30]:
# Line by line into the list
with open('seq.ppairs', 'r') as res_file_handler:
        content =  res_file_handler.readlines()

In [31]:
content

['% NUPACK 3.1.0\n',
 '% Program: pairs\n',
 '% Start time: Tue Apr 25 23:16:58 2017\n',
 '% Command: pairs -T 37 seq \n',
 '% Minimum printed pair probability: 0.001\n',
 '% Sequence:  TACTGATATACCTCTCCGTTAAGGATACGCCGTAAGCTCCATTGGTCACA\n',
 '% v(pi): 1\n',
 '% Parameters: RNA, 1995\n',
 '% Dangles setting: 1\n',
 '% Temperature (C): 37.0\n',
 '% Sodium concentration: 1.0000 M\n',
 '% Magnesium concentration: 0.0000 M\n',
 '% Free energy: -6.69513088e+00 kcal/mol\n',
 '\n',
 '50\n',
 '1\t6\t1.1797e-02\n',
 '1\t8\t1.4809e-02\n',
 '1\t10\t1.3374e-02\n',
 '1\t21\t2.5288e-03\n',
 '1\t34\t9.5238e-03\n',
 '2\t7\t1.4372e-02\n',
 '2\t9\t1.5879e-02\n',
 '2\t19\t2.3056e-02\n',
 '2\t20\t2.6685e-03\n',
 '2\t33\t1.1627e-02\n',
 '2\t46\t2.4771e-02\n',
 '3\t18\t2.6392e-02\n',
 '3\t23\t2.1177e-03\n',
 '3\t24\t2.2267e-03\n',
 '3\t32\t1.1675e-02\n',
 '3\t36\t1.9829e-03\n',
 '3\t45\t2.7716e-02\n',
 '4\t8\t2.7102e-03\n',
 '4\t10\t1.2572e-02\n',
 '4\t18\t1.6616e-03\n',
 '4\t21\t1.7369e-02\n',
 '4\t22\t2.05

According to the manual, the output is formatted as following:
* First comes commented header with parameters of secondary structure assembly
* Empty line
* Integer identifying total number of nucleotide N
* Probabilities of each nucleotide pairing with other nucleotide.
    * They are numbered starting from "1"
    * 3 columns: nucleotide number (1st column), its pair number (2nd column), probability (3rd column)
    * Number N+1 is unpaired nucleotide (looks like, I'm not 100% sure now).
    * There is a cutoff value for probability. Pairs with probabilities below cutoff will not show up.

## Calculating partition function

In [32]:
# -T 37 is how one specifies temperature
!pfunc -T 37 seq

% NUPACK 3.1.0
% Program: pfunc
% Start time: Tue Apr 25 23:16:58 2017
% Command: pfunc -T 37 seq 
% Sequence:  TACTGATATACCTCTCCGTTAAGGATACGCCGTAAGCTCCATTGGTCACA
% v(pi): 1
% Parameters: RNA, 1995
% Dangles setting: 1
% Temperature (C): 37.0
% Sodium concentration: 1.0000 M
% Magnesium concentration: 0.0000 M
%
% Free energy (kcal/mol) and partition function:
-6.69513088e+00
5.22117134271103e+04


In [40]:
result = !pfunc -T 37 seq

In [41]:
result

['% NUPACK 3.1.0',
 '% Program: pfunc',
 '% Start time: Tue Apr 25 23:24:27 2017',
 '% Command: pfunc -T 37 seq ',
 '% Sequence:  UACUGAUAUACCUCUCCGUUAAGGAUACGCCGUAAGCUCCAUUGGUCACA',
 '% v(pi): 1',
 '% Parameters: RNA, 1995',
 '% Dangles setting: 1',
 '% Temperature (C): 37.0',
 '% Sodium concentration: 1.0000 M',
 '% Magnesium concentration: 0.0000 M',
 '%',
 '% Free energy (kcal/mol) and partition function:',
 '-6.69513088e+00',
 '5.22117134271103e+04']

## Checking if T-U replacement makes any effect

In [34]:
import re

In [38]:
# Saving sample sequence to input file
seqU = re.sub('T', 'U', seq)
SeqToFile(seqU)

In [39]:
# You can also input just a sequence, without saving it to hard drive
!pfunc -T 37 seq

% NUPACK 3.1.0
% Program: pfunc
% Start time: Tue Apr 25 23:19:18 2017
% Command: pfunc -T 37 seq 
% Sequence:  UACUGAUAUACCUCUCCGUUAAGGAUACGCCGUAAGCUCCAUUGGUCACA
% v(pi): 1
% Parameters: RNA, 1995
% Dangles setting: 1
% Temperature (C): 37.0
% Sodium concentration: 1.0000 M
% Magnesium concentration: 0.0000 M
%
% Free energy (kcal/mol) and partition function:
-6.69513088e+00
5.22117134271103e+04


When doing RNA and keeping T, the deltaG = -6.695

When RNA with U instead of T, the deltaG = -6.695

There is no difference, seems Nupack automatically replaces T with U.